# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [753]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv
import panel as pn
from panel.interact import interact
from panel import widgets

In [754]:
# Read the Mapbox API key
load_dotenv()
mapbox_token = os.getenv("MAPBOX_API_KEY")
px.set_mapbox_access_token(mapbox_token)

# Import Data

In [755]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [756]:
sfo_data.isnull().sum()

neighborhood           0
sale_price_sqr_foot    5
housing_units          0
gross_rent             0
dtype: int64

In [757]:
sfo_data = sfo_data.dropna()
sfo_data.isnull().sum()

neighborhood           0
sale_price_sqr_foot    0
housing_units          0
gross_rent             0
dtype: int64

In [758]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""

    # YOUR CODE HERE!
    mean_housing_units = sfo_data.groupby(['year']).mean().iloc[: , 1]
    miny = mean_housing_units.min()
    maxy = mean_housing_units.max()
    std = mean_housing_units.std()
    ymin = (miny-std) 
    ymax = (maxy+std)
    
    return mean_housing_units.hvplot.bar(xlabel='Year', ylabel='Housing Units', ylim=([ymin, ymax]), title='Average Annual Housing Units in San Francisco').opts(yformatter="%.0f")

In [759]:
def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    
    # YOUR CODE HERE!
    avg_gross_rent_ppsqft = sfo_data[['sale_price_sqr_foot', 'gross_rent']].groupby(['year']).mean()
    
    return avg_gross_rent_ppsqft['gross_rent'].hvplot.line(title='Average Annual Gross Rent in San Francisco', xlabel='Year', ylabel='Gross Rent')


In [760]:
def average_sales_price():
    """Average Sales Price Per Year."""
    
    # YOUR CODE HERE!
    avg_gross_rent_ppsqft = sfo_data[['sale_price_sqr_foot', 'gross_rent']].groupby(['year']).mean()
    
    return avg_gross_rent_ppsqft['sale_price_sqr_foot'].hvplot.line(title='Average Sale Price Per Square Foot in San Francisco', xlabel='Year', ylabel='Avg. Price per Square Foot')

In [761]:
def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    
    # YOUR CODE HERE!
    neighborhoods_avg_ppsqft = sfo_data.groupby(['year','neighborhood']).mean()[['sale_price_sqr_foot']]
    
    return neighborhoods_avg_ppsqft.hvplot.line(groupby='neighborhood', xlabel='Year', ylabel='Avg. Sale Price per Square Foot')
    
    

In [762]:
def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    
    # YOUR CODE HERE!
    top_10_most_expensive_neighborhoods = sfo_data.groupby(['neighborhood']).mean().sort_values('sale_price_sqr_foot', ascending=False).iloc[0:10,0]
    miny = top_10_most_expensive_neighborhoods.min()
    maxy = top_10_most_expensive_neighborhoods.max()
    std = top_10_most_expensive_neighborhoods.std()
    ymin = (miny-std) 
    ymax = (maxy+(std*3))
    return top_10_most_expensive_neighborhoods.hvplot.bar(title='Top 10 Expensive Neighborhoods in SF', xlabel='Neighborhood', ylabel='Avg. Sale Price per Square Foot', rot=90, width=600, height=600)

In [763]:
def parallel_coordinates():
    """Parallel Coordinates Plot."""
    
    # YOUR CODE HERE!
    avg_neighborhood_values = sfo_data.groupby(['neighborhood']).mean().sort_values('sale_price_sqr_foot', ascending=False).iloc[0:10].reset_index()
    
    return px.parallel_coordinates(avg_neighborhood_values, color='sale_price_sqr_foot')


    

In [764]:
def parallel_categories():
    """Parallel Categories Plot."""
    
    # YOUR CODE HERE!
    avg_neighborhood_values = sfo_data.groupby(['neighborhood']).mean().sort_values('sale_price_sqr_foot', ascending=False).iloc[0:10].reset_index()
   
    return px.parallel_categories(avg_neighborhood_values, dimensions=['neighborhood', 'sale_price_sqr_foot', 'housing_units', 'gross_rent'], color='sale_price_sqr_foot', color_continuous_scale = px.colors.sequential.Inferno)
    
    

In [765]:
def neighborhood_map():
    """Neighborhood Map"""
    
    # YOUR CODE HERE!
    avg_neighborhood_values = sfo_data.reset_index().groupby(['neighborhood']).mean()
    avg_neighborhood_values = avg_neighborhood_values.reset_index()
    neighborhood_values_locations = pd.concat([df_neighborhood_locations, avg_neighborhood_values], axis=1)
    neighborhood_values_locations = neighborhood_values_locations.drop(columns=['neighborhood'])
    mapbox_plot_gross_rent_ppsqft = px.scatter_mapbox(neighborhood_values_locations, lat='Lat', lon='Lon', color='gross_rent', size='sale_price_sqr_foot', title='Average Sale Price per Square Foot and Gross Rent in San Francisco', zoom=10)
    return mapbox_plot_gross_rent_ppsqft

    

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [766]:
# YOUR CODE HERE!
title = '## Real Estate Analysis: San Francisco 2010-2016'

welcome_tab = pn.Column(pn.Column(title), neighborhood_map())

yearly_market_analysis_tab = pn.Row(housing_units_per_year(), average_gross_rent(), average_sales_price())

neighborhood_analysis_tab = pn.Column(average_price_by_neighborhood(), top_most_expensive_neighborhoods())

parallel_plots_tab = pn.Column(parallel_coordinates(), parallel_categories())

In [767]:
SFO_Real_Estate_Analysis_Dashboard = pn.Tabs(
    ("Welcome", welcome_tab),
    ("Yearly Market Analysis", yearly_market_analysis_tab),
    ("Neighborhood Analysis", neighborhood_analysis_tab),
    ("Parallel Plots Analysis", parallel_plots_tab))

SFO_Real_Estate_Analysis_Dashboard

Tabs
    [0] Column
        [0] Column
            [0] Markdown(str)
        [1] Plotly(Figure)
    [1] Row
        [0] HoloViews(Bars)
        [1] HoloViews(Curve)
        [2] HoloViews(Curve)
    [2] Column
        [0] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()
        [1] HoloViews(Bars)
    [3] Column
        [0] Plotly(Figure)
        [1] Plotly(Figure)

## Serve the Panel Dashboard

In [768]:
SFO_Real_Estate_Analysis_Dashboard.servable()

Tabs
    [0] Column
        [0] Column
            [0] Markdown(str)
        [1] Plotly(Figure)
    [1] Row
        [0] HoloViews(Bars)
        [1] HoloViews(Curve)
        [2] HoloViews(Curve)
    [2] Column
        [0] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()
        [1] HoloViews(Bars)
    [3] Column
        [0] Plotly(Figure)
        [1] Plotly(Figure)